Theory:

Based on animals population in an ecosystem.

Habitat Suitability index - > if population is high, HSI is high. This is the objective function. 

Probability of migration is high in high population ecosystem. THe less population, the high migratibility. 

Immigration (to) import (I)

Emigration (from) export (E)

Smax is the highest HSI

Emigration Rate (Mu)

Immigration Rate (Lambda)

SIV -> Suitability index Variable - > like chromosomes in Genetic Algorithm.

BBO Algorithm Steps:

        1- Produce a set of randomized biography population (Initial population) and sort.
        2- Determine mu and lambda based on rank of biography population.
        3- For each biography place repeat:
                4- for each SIV like k in biography place i do:
                        5- with lambda i prob, changes will apply in xik.
                                6- Determine origin of immigration by mu and randomized -> j.
                                7- do the immigration from xjk to xik.
                                8- with certain probability mutation will be applied on xik.

        9- Evaluate mew achieved biography places.
        10- Merge original population and migrated driven population and create new population for new iteration (Merge,Sort and Truncate populations, or predefined shares.)
        11- If end criteria is not met yet, return to the step 3.



In [16]:
#Import libraries
import numpy as np
import numpy.matlib
import pandas as pd

In [29]:
#function definition to be optimized
def sphere(x):
    return sum(np.power(x,2))

In [47]:
#Problem Definition

varSize = 5     #number of decision variables
varMin = -10    #decision variable lower bound
varMax = 10     #decision variable upper bound

def costFunction(x):
    return sphere(x)


In [7]:
#BBo Parameters
maxIt = 100     #maximum number od iteration
nPop = 50       #number of habitats (population size)
keepRate = 0.2  #Keep Rate
nKeep = round(keepRate * nPop,0)  #number of kept habitats
nNew = nPop - nKeep     #Number of new habitats

#Migration Rate
mu = np.linspace(1,0,nPop)  #Emigration Rates
lambdas = 1- mu     #immigration Rates

alpha = 0.9
pMutation = 0.1
sigma = 0.02 * (varMax - varMin)


In [51]:
#Initialization

#Habitat
popHabitat_positions  = []
popHabitat_costs  = []
for i in range(nPop):
    position = np.random.uniform(varMin,varMax,varSize).tolist()
    popHabitat_positions.append(position)

    cost = costFunction(position)
    popHabitat_costs.append(cost)

[7.3389426783584035,
 -7.206786903682561,
 4.04286863060727,
 -6.798268395104811,
 -4.736087762015737]